# 보팅 앙상블
단일 모델을 앙상블하여 더 나은 예측을 하는 앙상블 모델을 만들어 보겠습니다.

In [426]:
from sklearn import datasets
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 손글씨 데이터 로드

In [501]:
mnist = datasets.load_digits()
features, labels = mnist.data, mnist.target
X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.2)

# 단일 모델 정확도 측정
의사결정트리, knn, svm 모델의 정확도를 측정합니다.

In [503]:
dtree = tree.DecisionTreeClassifier(criterion="gini", max_depth=8, max_features=32,random_state=35)
dtree = dtree.fit(X_train, y_train)
dtree_predicted = dtree.predict(X_test)

knn = KNeighborsClassifier(n_neighbors=135).fit(X_train, y_train)
knn_predicted = knn.predict(X_test)

svm = SVC(C=0.15, gamma=0.003,probability=True,random_state=35).fit(X_train, y_train)
svm_predicted = svm.predict(X_test)

In [504]:
print("[accuarcy]")
print("d-tree: ",accuracy_score(y_test, dtree_predicted))
print("knn   : ",accuracy_score(y_test, knn_predicted))
print("svm   : ",accuracy_score(y_test, svm_predicted))

[accuarcy]
d-tree:  0.825
knn   :  0.9111111111111111
svm   :  0.9361111111111111


소프트 보팅 또는 하드 보팅은 sklearn의 voting classifier를 사용하여 쉽게 구현할 수 있습니다.  
직접 소프트 보팅을 구현하실 때는 predict_proba 함수를 사용하여 테스트 수행 시 측정된 분류값 별 확률을 사용하시면 됩니다.  
아래 3개의 MNIST 테스트 데이터0부터 9까지의 확률을 보실 수 있습니다.

In [505]:
dtree_proba = dtree.predict_proba(X_test)
print(dtree_proba[0:3])

[[0.         0.00806452 0.00806452 0.96774194 0.         0.
  0.         0.         0.00806452 0.00806452]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]]


In [499]:
voting_clf = VotingClassifier(estimators=[
    ('decision_tree', clf), ('knn', knn), ('svm', svm)], 
    weights=[1,1,1], voting='hard').fit(X_train, y_train)
ensemble_predicted = voting_clf.predict(X_test)
accuracy_score(y_test, ensemble_predicted)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9027777777777778

In [500]:
voting_clf = VotingClassifier(estimators=[
    ('decision_tree', clf), ('knn', knn), ('svm', svm)], 
    weights=[1,1,1], voting='soft').fit(X_train, y_train)
ensemble_predicted = voting_clf.predict(X_test)
accuracy_score(y_test, ensemble_predicted)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.925